# REVIEWS
- `main_legs` and `start_legs` issue 
    - may need to generate the `start_legs` for every trips due to the shuttle trip issue of start_place
- Whether any minimum number of trips needed . Issue with shuttle trip 



### SET UP AND IMPORTS 

In [22]:
# SET UP AND IMPORTS 
import pandas as pd 
import datetime
import numpy as np 

df = pd.read_excel('dataset/processed/psl.xlsx')
# Adding the Terminal Gap and Duty only for processing 
df['Terminal Gap'] = [0] * len(df)
df['Duty'] = [0] * len(df)

START_PLACE = 'PSL'
#start_legs contain the legs with departure place as start_place
start_legs = df[df['Departure Place'] == START_PLACE].sort_values('Departure Time', ascending=True)
#main_legs contains all other legs 
main_legs = df[df['Departure Place'] != START_PLACE]
MIN_DUTY = 27000 #setting the minimum duty as 7:00 hrs 
# The ideal duty is 7:30 hrs with a 15 min break at the start and end . Then total 8:00 
MAX_DUTY = 30600 # setting the maximum duty as 8:30 hrs
MIN_TERMINAL_GAP = 300
MAX_TERMINAL_GAP = 900

### SUPPORTING FUNCTIONS 

In [23]:
#SUPPORTING FUNCTIONS 
def generateTempSet(leg):
    # Creating a copy of the main legs
    temp_set = df[(df['Departure Place'] == leg['Arrival Place']) & (df['Departure Time'] > leg['Arrival Time act'])]
    temp_set = temp_set.sort_values('Departure Time', ascending=True).reset_index(drop=True)
    temp_set['Terminal Gap'] = temp_set['Departure Time'] - leg['Arrival Time act']
    # Sorting and resetting index
    temp_set['Duty'] = leg['Duty'] + temp_set['Running Time'] + temp_set['Terminal Gap']
    # display(temp_set)
    temp_set = temp_set[
        (temp_set['Terminal Gap'] >= MIN_TERMINAL_GAP) & (temp_set['Terminal Gap'] < MAX_TERMINAL_GAP) #|
        #(temp_set['Duty'] >= MIN_DUTY) & (temp_set['Duty'] <= MAX_DUTY)  
    ]
    return temp_set

generateTempSet(start_legs.iloc[0])

def popTempSet(temp_set) -> pd.DataFrame:
    temp_set.drop(0, inplace= True)
    temp_set.reset_index(inplace=True)
    return temp_set.drop('index',axis= 1)


def displayTrip(stack):
    frame = pd.DataFrame()
    for x in stack:
        frame = pd.concat([frame, (x['current_leg'].to_frame()).T], ignore_index=True)
    frame['Terminal Gap'] = frame['Terminal Gap'].shift(-1)
    duty = second=frame['Duty'].iloc[len(frame) - 1]
    hrs, rem = divmod(duty, 3600)
    mint, sec = divmod(rem , 60)
    duty = datetime.time(hour=hrs, minute=mint, second= sec)
    frame = frame.drop('Duty', axis=1)
    return [frame, duty]

# def seconds_to_time(df, columns):
#     df[columns] = df[columns].apply(lambda x: pd.to_datetime(x, unit='s').dt.strftime('%H:%M:%S'))

### ALGORITHM

In [26]:
# ALGORITHM
def generateTrip(start_leg):
    stack = [{"current_leg": start_leg, "temp_set": pd.DataFrame()}] # contain the legs in the trip 
    temp_set = generateTempSet(start_leg)
    stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})
    while len(stack) > 1 : # and stack[-1]['current_leg']['Arrival Place'] != START_PLACE
        temp_set = generateTempSet(stack[-1]['current_leg'])
        if temp_set.empty :
            stack.pop()
            while(stack[-1]['temp_set'].empty) :
                    stack.pop()
            stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
            stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])
            # print("Got null 11")
            # print(stack)
            # display(displayTrip(stack))
            continue
        stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})
        if (top_leg:=stack[-1]['current_leg'])['Arrival Place'] == START_PLACE:
            if MIN_DUTY > top_leg['Duty']:
                # NEED TO ADDRESS SHUTTLE TRIP ISSUE OF START_PLACE
                while(stack[-1]['temp_set'].empty) :
                    stack.pop()
                stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
                stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

            elif top_leg['Duty'] > MAX_DUTY:
                stack.pop()
                while(stack[-1]['temp_set'].empty) :
                    stack.pop()
                stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
                stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

            else: # MIN_DUTY <= top_leg['Duty'] <= MAX_DUTY 
                #ideal case 
                break;
    result = displayTrip(stack)
    display(result[0])
    print("Steering Duty = ", result[1])

error = 0
for i in range(0, len(start_legs)):
    try:
        generateTrip(start_legs.iloc[i])
    except Exception as e:
        error += 1
        continue

print(f"Total trips = {len(start_legs) - error} ..... Missing {error} routes  ")
    

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,13200,PSL,NH,KLKV,13500,3.5,300,300
1,124,13800,KLKV,NH,TVM,17700,33.7,3900,600
2,313,18300,TVM,NH-UDA,KNVLA,21900,32.0,3600,600
3,641,22500,KNVLA,UDA,MC,28800,39.0,6300,600
4,341,29400,MC,PTM,NTA,33000,27.7,3600,600
5,200,33600,NTA,NH,TVM,36600,20.7,3000,300
6,376,36900,TVM,NH,KLKV,41400,33.7,4500,300
7,66,41700,KLKV,NH,PSL,42300,3.5,600,None


Steering Duty =  08:00:00


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,6,18300,PSL,NH,KLKV,18900,3.5,600,300
1,501,19200,KLKV,KRKM,VLRD,21600,17.0,2400,600
2,542,22200,VLRD,KRKM,KLKV,24600,17.0,2400,600
3,505,25200,KLKV,KRKM,VLRD,27600,17.0,2400,600
4,547,28200,VLRD,KRKM,KLKV,30600,17.0,2400,600
5,189,31200,KLKV,NH,MC,38400,40.0,7200,300
6,649,38700,MC,NTA-MJ-KRKM,PSL,45900,46.0,7200,None


Steering Duty =  07:30:00


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,17,18600,PSL,NH,KLKV,18900,3.5,300,300
1,501,19200,KLKV,KRKM,VLRD,21600,17.0,2400,600
2,542,22200,VLRD,KRKM,KLKV,24600,17.0,2400,600
3,505,25200,KLKV,KRKM,VLRD,27600,17.0,2400,600
4,547,28200,VLRD,KRKM,KLKV,30600,17.0,2400,600
5,189,31200,KLKV,NH,MC,38400,40.0,7200,300
6,649,38700,MC,NTA-MJ-KRKM,PSL,45900,46.0,7200,None


...

Steering Duty =  07:50:00


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,62,52500,PSL,NH,KLKV,53100,3.5,600,600
1,254,53700,KLKV,NH,CSTN,61200,42.0,7500,600
2,453,61800,CSTN,NH,KLKV,69600,42.0,7800,600
3,309,70200,KLKV,NH,TVM,75000,33.7,4800,600
4,498,75600,TVM,NH,KLKV,80100,33.7,4500,300
5,112,80400,KLKV,NH,PSL,81000,3.5,600,None


Steering Duty =  07:45:00


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,63,52800,PSL,NH,KLKV,53400,3.5,600,300
1,254,53700,KLKV,NH,CSTN,61200,42.0,7500,600
2,453,61800,CSTN,NH,KLKV,69600,42.0,7800,600
3,309,70200,KLKV,NH,TVM,75000,33.7,4800,600
4,498,75600,TVM,NH,KLKV,80100,33.7,4500,300
5,112,80400,KLKV,NH,PSL,81000,3.5,600,None


Steering Duty =  07:40:00
Total trips = 55 ..... Missing 49 routes  


### MAIN PART

In [25]:
# MAIN


